In [4]:
import os
import glob
import pandas as pd
import numpy as np
import qiime2 as q2

# dist eval
from skbio import DistanceMatrix
from skbio.stats.distance import permanova, permdisp, bioenv

# KNN eval
from inspect import signature
from sklearn import preprocessing
from sklearn import svm, datasets
from skbio import OrdinationResults
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import recall_score, precision_score, roc_auc_score, r2_score, mean_squared_error


In [2]:
metrics = {'Aitchison-UMAP':'aitchison-umap-distance_matrix.qza',
           'Euclidean-UMAP':'euclidean-umap-distance_matrix.qza',
           'Jaccard-UMAP': 'jaccard-umap-distance_matrix.qza',
           'Bray-Curtis-UMAP': 'braycurtis-umap-distance_matrix.qza',
           'UniFrac-UMAP': 'unweighted-unifrac-umap-distance_matrix.qza',
           'W-UniFrac-UMAP': 'weighted-unifrac-umap-distance_matrix.qza',
           'Aitchison-UMAP-NN80':'aitchison-umap-nn50-distance_matrix.qza',
           'Euclidean-UMAP-NN80':'euclidean-umap-nn50-distance_matrix.qza',
           'Jaccard-UMAP-NN80': 'jaccard-umap-nn50-distance_matrix.qza',
           'Bray-Curtis-UMAP-NN80': 'braycurtis-umap-nn50-distance_matrix.qza',
           'UniFrac-UMAP-NN80': 'unweighted-unifrac-umap-nn50-distance_matrix.qza',
           'W-UniFrac-UMAP-NN80': 'weighted-unifrac-umap-nn50-distance_matrix.qza',
           'Aitchison': 'aitchison-mds-distance_matrix.qza',
           'Euclidean': 'euclidean-mds-distance_matrix.qza',
           'Jaccard':'jaccard_distance_matrix.qza',
           'Bray-Curtis':'bray_curtis_distance_matrix.qza',
           'UniFrac':'unweighted_unifrac_distance_matrix.qza',
           'W-UniFrac':'weighted_unifrac_distance_matrix.qza'}
eval_col = {'keyboard':'host_subject_id',
            'soils':'ph'}

perm_res = {}
for data_set in ['keyboard', 'soils']:
    mf_ = pd.read_csv(glob.glob(os.path.join('data', data_set, '*.txt'))[0],
                      sep='\t', index_col=0)
    for mname_, metric in metrics.items():
        dist_ = q2.Artifact.load(os.path.join('results', data_set, metric))
        dist_ = dist_.view(DistanceMatrix)
        if data_set == 'keyboard':
            perm_tmp = pd.DataFrame(permanova(dist_, mf_.loc[dist_.ids,
                                                             eval_col[data_set]])).T
            # save them
            perm_res[(data_set, mname_)] = perm_tmp[['test statistic']]
        elif data_set == 'soils':
            mf_[eval_col[data_set]] = mf_[eval_col[data_set]].astype(int)
            perm_tmp = pd.DataFrame(bioenv(dist_, mf_.loc[dist_.ids,
                                                             [eval_col[data_set]]]))
            # save them
            perm_res[(data_set, mname_)] = perm_tmp[['correlation']].rename({'correlation':'test statistic'},
                                                                            axis=1)

perm_res_df = pd.concat(perm_res).reset_index()
perm_res_df = perm_res_df.rename({'level_0':'dataset',
                                  'level_1':'method',
                                   'vars':'Eval'}, axis=1)
perm_res_df = perm_res_df.replace({'PERMANOVA results':'PERMANOVA',
                                   'ph':'BIO-ENV'}, regex=True)
perm_res_df = perm_res_df.set_index(['dataset','method'])
perm_res_df.to_csv('results/eval-tables/distance-stats.tsv', sep='\t')
perm_res_df        

Eval  test statistic
dataset  method                                          
keyboard Aitchison-UMAP         PERMANOVA     2944.506131
         Euclidean-UMAP         PERMANOVA      141.118716
         Jaccard-UMAP           PERMANOVA     4095.255652
         Bray-Curtis-UMAP       PERMANOVA      788.949658
         UniFrac-UMAP           PERMANOVA     1714.799930
         W-UniFrac-UMAP         PERMANOVA      612.321876
         Aitchison-UMAP-NN80    PERMANOVA      490.195901
         Euclidean-UMAP-NN80    PERMANOVA      136.533268
         Jaccard-UMAP-NN80      PERMANOVA      513.868837
         Bray-Curtis-UMAP-NN80  PERMANOVA     3317.102278
         UniFrac-UMAP-NN80      PERMANOVA      284.233357
         W-UniFrac-UMAP-NN80    PERMANOVA      443.792181
         Aitchison              PERMANOVA       25.519420
         Euclidean              PERMANOVA       86.802779
         Jaccard                PERMANOVA       10.741681
         Bray-Curtis            PERMANOVA       64.557365
         UniFrac                PERMANOVA       16.380505
         W-UniFrac              PERMANOVA       92.905644
soils    Aitchison-UMAP           BIO-ENV        0.713596
         Euclidean-UMAP           BIO-ENV        0.674822
         Jaccard-UMAP             BIO-ENV        0.711920
         Bray-Curtis-UMAP         BIO-ENV        0.701114
         UniFrac-UMAP             BIO-ENV        0.745139
         W-UniFrac-UMAP           BIO-ENV        0.697162
         Aitchison-UMAP-NN80      BIO-ENV        0.739647
         Euclidean-UMAP-NN80      BIO-ENV        0.612586
         Jaccard-UMAP-NN80        BIO-ENV        0.752865
         Bray-Curtis-UMAP-NN80    BIO-ENV        0.733704
         UniFrac-UMAP-NN80        BIO-ENV        0.772459
         W-UniFrac-UMAP-NN80      BIO-ENV        0.697394
         Aitchison                BIO-ENV        0.588118
         Euclidean                BIO-ENV        0.414573
         Jaccard                  BIO-ENV        0.699039
         Bray-Curtis              BIO-ENV        0.694536
         UniFrac                  BIO-ENV        0.701726
         W-UniFrac                BIO-ENV        0.675978

In [20]:
metrics = {'Aitchison-UMAP':'aitchison-umap-pcoa_results.qza',
           'Euclidean-UMAP':'euclidean-umap-pcoa_results.qza',
           'Jaccard-UMAP': 'jaccard-umap-pcoa_results.qza',
           'Bray-Curtis-UMAP': 'braycurtis-umap-pcoa_results.qza',
           'UniFrac-UMAP': 'unweighted-unifrac-umap-pcoa_results.qza',
           'W-UniFrac-UMAP': 'weighted-unifrac-umap-pcoa_results.qza',
           'Aitchison-UMAP-NN80':'aitchison-umap-nn50-pcoa_results.qza',
           'Euclidean-UMAP-NN80':'euclidean-umap-nn50-pcoa_results.qza',
           'Jaccard-UMAP-NN80': 'jaccard-umap-nn50-pcoa_results.qza',
           'Bray-Curtis-UMAP-NN80': 'braycurtis-umap-nn50-pcoa_results.qza',
           'UniFrac-UMAP-NN80': 'unweighted-unifrac-umap-nn50-pcoa_results.qza',
           'W-UniFrac-UMAP-NN80': 'weighted-unifrac-umap-nn50-pcoa_results.qza',
           # 'Euclidean-UMAP-subsampled': 'euclidean-umap-rarefy500-pcoa_results.qza',
           'Aitchison': 'aitchison-mds-pcoa_results.qza',
           'Euclidean': 'euclidean-mds-pcoa_results.qza',
           'Jaccard':'jaccard_pcoa_results.qza',
           'Bray-Curtis':'bray_curtis_pcoa_results.qza',
           'UniFrac':'unweighted_unifrac_pcoa_results.qza',
           'W-UniFrac':'weighted_unifrac_pcoa_results.qza'}
# metrics = {'AU-MAP':'aitchison-umap-pcoa_results.qza',
#            'U-MAP':'euclidean-umap-pcoa_results.qza',
#            'A-PCoA': 'aitchison-mds-pcoa_results.qza',
#            'U-MAP-rarefy': 'euclidean-umap-rarefy500-pcoa_results.qza',
#            'Jaccard':'jaccard_pcoa_results.qza',
#            'Bray-Curtis':'bray_curtis_pcoa_results.qza',
#            'UniFrac':'unweighted_unifrac_pcoa_results.qza',
#            'W-UniFrac':'weighted_unifrac_pcoa_results.qza'}
eval_col = {'keyboard':'host_subject_id',
            'soils':'ph'}

keyboard_n_knn_components = 2
soil_n_knn_components = 1

ML_res = {}
for data_set in ['keyboard', 'soils']:
    mf_ = pd.read_csv(glob.glob(os.path.join('data', data_set, '*.txt'))[0],
                      sep='\t', index_col=0)
    for mname_, metric in metrics.items():
        ord_ = q2.Artifact.load(os.path.join('results', data_set, metric))
        ord_ = ord_.view(OrdinationResults).samples
        
        if data_set == 'keyboard':

            X = ord_[list(range(keyboard_n_knn_components))].values
            y = mf_.loc[ord_.index, eval_col[data_set]].replace({'M9':'M3'},
                                                                regex=True)

            le = preprocessing.LabelEncoder()
            le.fit(y)
            y = le.transform(y) 

            scoring_ = {'APR':'average_precision',
                        'AUC':'roc_auc'}

            for scoring_method, scoring_ in scoring_.items():

                # instantiate the grid
                k_range = list(range(1, 60))
                param_grid = dict(n_neighbors=k_range)
                param_grid['weights'] = ['uniform','distance']
                # param_grid['p'] = [1, 2]
                # param_grid['algorithm'] = ['ball_tree', 'kd_tree', 'brute']
                classifier = KNeighborsClassifier()
                grid = GridSearchCV(classifier, param_grid, cv=5,
                                    scoring=scoring_)
                grid.fit(X, y)
                classifier = grid.best_estimator_

                # for i, score_ in enumerate(grid.cv_results_['mean_test_score']):
                for i, (score_, weights, k) in enumerate(
                        zip(grid.cv_results_['mean_test_score'],
                            grid.cv_results_['param_weights'].data,
                            grid.cv_results_['param_n_neighbors'].data)
                        ):
                    ML_res[(data_set, 'KNN-Classifier', scoring_method, weights, k, mname_)] = [score_]
                    # ML_res[(data_set, 'KNN-Classifier', scoring_method, mname_)] = [score_]

        elif data_set == 'soils':
            
            X = ord_[list(range(soil_n_knn_components))].values
            y = mf_.loc[ord_.index,
                        eval_col[data_set]].values

            scoring_ = {'$R^{2}$':'r2',
                        'Neg-MSE':'neg_mean_squared_error'}

            for scoring_method, scoring_ in scoring_.items():

                # instantiate the grid
                k_range = list(range(1, 69))
                param_grid = dict(n_neighbors=k_range)
                param_grid['weights'] = ['uniform','distance']
                # param_grid['p'] = [1, 2]
                # param_grid['algorithm'] = ['ball_tree', 'kd_tree', 'brute']
                classifier = KNeighborsRegressor()
                grid = GridSearchCV(classifier, param_grid, cv=5,
                                    scoring=scoring_)
                grid.fit(X, y)
                classifier = grid.best_estimator_

                for i, (score_, weights, k) in enumerate(
                        zip(grid.cv_results_['mean_test_score'],
                            grid.cv_results_['param_weights'].data,
                            grid.cv_results_['param_n_neighbors'].data)
                        ):
                    # print((data_set, 'KNN-Classifier', scoring_method, mname_))
                    ML_res[(data_set, 'KNN-Classifier', scoring_method, weights, k, mname_)] = [score_]

ML_res = pd.DataFrame(ML_res, ['value']).T
ML_res.index.names = ['dataset','KNN','Eval','weights','k','method']
ML_res.to_csv('results/eval-tables/KNN-stats-total.tsv', sep='\t')
# idx = ML_res.groupby(['dataset','KNN','Eval', 'method', 'weights']).transform(np.argmax)
idx = ML_res.groupby(['dataset','KNN','Eval', 'method']).transform(np.argmax)
max_ML_res = ML_res[(idx['value'].values == ML_res.index)]
max_ML_res.to_csv('results/eval-tables/KNN-stats.tsv', sep='\t')
max_ML_res

/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Deprec

/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Deprec

/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/garmstro/miniconda3/envs/qiime2-2019.10-dev/lib/python3.6/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.ar

value
dataset  KNN            Eval    weights  k  method                  
keyboard KNN-Classifier APR     uniform  1  Aitchison-UMAP  1.000000
                        AUC     uniform  1  Aitchison-UMAP  1.000000
                        APR     distance 24 Euclidean-UMAP  0.998767
                        AUC     distance 24 Euclidean-UMAP  0.997997
                        APR     uniform  1  Jaccard-UMAP    1.000000
...                                                              ...
soils    KNN-Classifier Neg-MSE uniform  6  Bray-Curtis    -0.296248
                        $R^{2}$ distance 22 UniFrac         0.873831
                        Neg-MSE distance 22 UniFrac        -0.206513
                        $R^{2}$ uniform  11 W-UniFrac       0.761989
                        Neg-MSE uniform  11 W-UniFrac      -0.391095

[72 rows x 1 columns]